# Adsorção usando VASP e ASE

Autor: [Prof. Elvis do A. Soares](https://github.com/elvissoares) 

Contato: [elvis@peq.coppe.ufrj.br](mailto:elvis@peq.coppe.ufrj.br) - [Programa de Engenharia Química, PEQ/COPPE, UFRJ, Brasil](https://www.peq.coppe.ufrj.br/)

---

Importando variáveis do VASP

In [1]:
import os
# Definindo o path para os arquivos de potencial de pseudopotenciais do VASP
# Certifique-se de que o caminho esteja correto para o seu sistema
os.environ['VASP_PP_PATH'] = '/home/elvis/Programs/vasp-6.5.1/pp'
os.environ['ASE_VASP_COMMAND'] = 'mpirun -np 1 vasp_std_gpu'
os.environ['NO_STOP_MESSAGE'] = '1' # to avoid warning from mpirun

# Importando o VASP calculator do ASE
from ase.calculators.vasp import Vasp

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from ase import Atoms, Atom
from ase.build import molecule

from ase.io import read, write
from ase.visualize import view

# Adsorção de O2 em Ag(111)

Ref: https://journals.aps.org/prb/pdf/10.1103/PhysRevB.65.075407

## Átomo de O

In [3]:
atom = Atoms([Atom('O', [0,0,0])])
atom.center(vacuum=4.0)       # ~8 Å box; 4 Å para cada lado, bom para isolar molécula
atom.pbc = True               # aplica condição de contorno periódica

In [4]:
view(atom, viewer='x3d')

In [5]:
calc = Vasp(directory='adsorcao/O',
            xc='PBE',                                   # funcional
            encut=450,                                  # safe default for PAW-PBE sets
            kpts=[1, 1, 1],gamma=True,                  # k-points
            ismear=0, sigma=0.05,                       # Gaussian smearing for molecules
            ediff=1e-6, ediffg=-0.02,                   # SCF and force criteria (~0.02 eV/Å)
            ibrion=2, isif=0, nsw=150,                  # CG ionic relax; fixed cell
            lreal='Auto',
            lwave=True, lcharg=True,lvtot=True,  # keep WAVECAR/CHGCAR/LOCPOT for post-processing
            atoms=atom)

In [6]:
E_O = atom.get_potential_energy()       

print('Energia do átomo de O = {:.3f} eV'.format(E_O))

Energia do átomo de O = -0.049 eV


## Molécula de O2

Otimização da geometria da molécula de O2

In [7]:
molecule = Atoms([Atom('O', [0, 0, -0.66]),
                  Atom('O', [0, 0, 0.66])])
molecule.center(vacuum=4.0)       # ~8 Å box; 4 Å para cada lado, bom para isolar molécula
molecule.pbc = True               # aplica condição de contorno periódica

In [8]:
calc = Vasp(directory='adsorcao/O2',
            xc='PBE',                                   # funcional
            encut=450,                                  # safe default for PAW-PBE sets
            kpts=[1, 1, 1],gamma=True,                  # k-points
            ismear=0, sigma=0.05,                       # Gaussian smearing for molecules
            ediff=1e-6, ediffg=-0.02,                   # SCF and force criteria (~0.02 eV/Å)
            ibrion=2, isif=0, nsw=150,                  # CG ionic relax; fixed cell
            lreal='Auto',
            lwave=True, lcharg=True,lvtot=True,  # keep WAVECAR/CHGCAR/LOCPOT for post-processing
            atoms=molecule)

In [9]:
E_O2 = molecule.get_potential_energy()       

print('Energia da molécula de O2 {:.3f} eV'.format(E_O2))

Energia da molécula de O2 -8.750 eV


$O_2 \to 2 O$

In [10]:
E_bind = 2 * E_O - E_O2

print("O2 -> 2 O --- E_bind = {0:1.3f} eV".format(E_bind))
print("(1/2)E_bind = {0:1.3f} eV".format(0.5*E_bind))

O2 -> 2 O --- E_bind = 8.652 eV
(1/2)E_bind = 4.326 eV


The experimental values are $1/2 E_b^{O_2}= 2.56$ eV, $d_0 = 2.28$ bohrs, and $\omega = 196$ meV (1580 $\text{cm}^{-1}$).

Ref: K. P. Huber and G. Herzberg, *Molecular Spectra and Molecular Structure IV: Constants of Diatomic Molecules* (Van Nostrand Reinhold, New York, 1979).

In [11]:
molecule.get_all_distances()

array([[0.        , 1.23523623],
       [1.23523623, 0.        ]])

In [12]:
view(molecule, viewer='x3d')

## Estado fundamental da molécula de O2

A molécula de O2 tem como estado fundamental um tripleto de spin. Assim, a molécula de O2 no estado fundamental possui dois elétrons desemparelhados. 

Correção de polarização de spin: `ISPIN` = 2

In [13]:
atom = Atoms([Atom('O', [0,0,0], magmom=2)])    # com momento magnético inicial
atom.center(vacuum=4.0)                        # ~8 Å box; 4 Å para cada lado, bom para isolar molécula
atom.pbc = True                                  # aplica condição de contorno periódica

calc = Vasp(directory='adsorcao/O-sp-triplet',
            xc='PBE',                                   # funcional
            encut=450,                                  # safe default for PAW-PBE sets
            kpts=[1, 1, 1],gamma=True,                  # k-points
            ismear=0, sigma=0.05,                       # Gaussian smearing for molecules
            ediff=1e-6, ediffg=-0.02,                   # SCF and force criteria (~0.02 eV/Å)
            ibrion=2, isif=0, nsw=150,                  # CG ionic relax; fixed cell
            ispin=2,                                    # turn spin-polarization on
            lreal='Auto',
            lwave=True, lcharg=True,lvtot=True,  # keep WAVECAR/CHGCAR/LOCPOT for post-processing
            atoms=atom)

E_O = atom.get_potential_energy()       

print('Energia do átomo de O = {:.3f} eV'.format(E_O))

print("Momento magnético do átomo de O = {0} Bohr magnetons".format(atom.get_magnetic_moment()))

Energia do átomo de O = -1.544 eV
Momento magnético do átomo de O = 1.9999968 Bohr magnetons


/home/elvis/Programs/miniforge3/envs/quantum/lib/python3.13/site-packages/ase/calculators/vasp/vasp.py:1004: UserWarning: Magnetic moment data not written in OUTCAR (LORBIT<10), setting magnetic_moments to zero.
Set LORBIT>=10 to get information on magnetic moments
  warn('Magnetic moment data not written in OUTCAR (LORBIT<10),'


In [14]:
molecule = Atoms([Atom('O', [0, 0, -0.66], magmom=1),
                  Atom('O', [0, 0, 0.66], magmom=1)]) # com momento magnético inicial
molecule.center(vacuum=4.0)                           # ~8 Å box; 4 Å para cada lado, bom para isolar molécula
molecule.pbc = True                                   # aplica condição de contorno periódica

calc = Vasp(directory='adsorcao/O2-sp-triplet',
            xc='PBE',                                   # funcional
            encut=450,                                  # safe default for PAW-PBE sets
            kpts=[1, 1, 1],gamma=True,                  # k-points
            ismear=0, sigma=0.05,                       # Gaussian smearing for molecules
            ediff=1e-6, ediffg=-0.02,                   # SCF and force criteria (~0.02 eV/Å)
            ibrion=2, isif=0, nsw=150,                  # CG ionic relax; fixed cell
            ispin=2,                                    # turn spin-polarization on
            lreal='Auto',
            lwave=True, lcharg=True,lvtot=True,  # keep WAVECAR/CHGCAR/LOCPOT for post-processing
            atoms=molecule)

E_O2 = molecule.get_potential_energy()       

print('Energia da molécula de O2 = {:.3f} eV'.format(E_O2))

print("Momento magnético da molécula de O2 = {0} Bohr magnetons".format(molecule.get_magnetic_moment()))

Energia da molécula de O2 = -9.856 eV
Momento magnético da molécula de O2 = 1.9999968 Bohr magnetons


Energia de ligação da molécula

In [15]:
E_bind = 2 * E_O - E_O2

print("O2 -> 2 O --- E_bind = {0:1.3f} eV".format(E_bind))

print("(1/2)E_bind = {0:1.3f} eV".format(0.5*E_bind))

O2 -> 2 O --- E_bind = 6.768 eV
(1/2)E_bind = 3.384 eV


In [16]:
molecule.get_all_distances()

array([[0.        , 1.23368751],
       [1.23368751, 0.        ]])

## Vibração da molécula de O2 e energia de ponto zero (ZPE)

Vimos que para uma molécula diatômica, podemos aproximar o movimento nuclear próximo do equilíbrio como um oscilador harmônico. A quantização dos estados de energia desse oscilador harmônico leva a um espectro de energia do tipo

$$E_n = \hbar \omega\left( n + \frac{1}{2}\right)$$

sendo $n$ um número quântico positivo e $\omega = \sqrt{k/m}$ a frequência angular de vibração da molécula. 

Usando o VASP, podemos calcular as frequências de vibração da molécula de O2. 


Para isso, vamos abrir um cálculo que já foi executado

In [17]:
calc = Vasp(restart=True, directory='adsorcao/O2-sp-triplet')

molecule = calc.get_atoms()

In [ ]:
# now do the vibrations
calc.set(directory='adsorcao/O2-sp-triplet-vib',
        ibrion=6,       # diferenças finitas para cálculo de vibração (with symmetry)
        nfree=2,        # usa diferenças centradas
        potim=0.02,     # tamanho do passo (em Å)
        nsw=1,
        atoms=molecule)

In [19]:
molecule.get_potential_energy()   

-9.84213327

In [20]:
vib_freq = calc.read_vib_freq()

for i, f in enumerate(vib_freq[0]):
    print("{0:02d}: {1} meV".format(i, f))

00: 194.114387 meV
01: 0.0 meV
02: 0.0 meV
03: 0.0 meV


Os valores experimentais são $1/2 E_b^{O_2}= 2.56$ eV, $d_0 = 2.28$ bohrs, and $\omega = 196$ meV (1580 $\text{cm}^{-1}$).

Ref: K. P. Huber and G. Herzberg, *Molecular Spectra and Molecular Structure IV: Constants of Diatomic Molecules* (Van Nostrand Reinhold, New York, 1979).

## Superfície Cristalina Ag(111)

In [72]:
from ase.build import (
    fcc100, fcc110, fcc111, fcc211, fcc111_root,
    bcc100, bcc110, bcc111, bcc111_root,
    hcp0001, hcp10m10, hcp0001_root,
    diamond100, diamond111
)

slab = fcc111("Ag", size=(2,2,3), vacuum=10.0,periodic=True)

print(slab)
print("Cell:", slab.get_cell())

Atoms(symbols='Ag12', pbc=True, cell=[[5.784133470105959, 0.0, 0.0], [2.8920667350529796, 5.009206523991598, 0.0], [0.0, 0.0, 24.722725201971137]], tags=...)
Cell: Cell([[5.784133470105959, 0.0, 0.0], [2.8920667350529796, 5.009206523991598, 0.0], [0.0, 0.0, 24.722725201971137]])


In [73]:
view(slab, viewer='x3d')

Para aumentar a velocidade do cálculo, nós congelaremos a superfície e permitiremos o adsorvato de relaxar.

In [74]:
from ase.constraints import FixAtoms

# now we constrain the slab
constraint = FixAtoms(mask=[atom.symbol=='Ag' for atom in slab])
slab.set_constraint(constraint)

In [75]:
calc = Vasp(directory='adsorcao/Ag-fcc111-slab',
            xc='PBE',
            kpts=[4,4,1],  # specifies k-points
            encut=450,
            lvtot=True,  # keep LOCPOT for post-processing
            atoms=slab)

In [76]:
E_slab = slab.get_potential_energy()

print("Energia do slab Ag(111) = {:.3f} eV".format(E_slab))

Energia do slab Ag(111) = -29.800 eV


In [77]:
view(slab, viewer='x3d')

# Adsorção

## 1. Átomo de O na posição FCC

Carregando a geometria do slab relaxada

In [78]:
calc = Vasp(restart=True, directory='adsorcao/Ag-fcc111-slab')


ads_system = fcc111("Ag", size=(2,2,3), vacuum=10.0,periodic=True)
ads_system.calc = calc

In [79]:
from ase.build import add_adsorbate

add_adsorbate(ads_system, "O", height=1.2, position="fcc")

Para aumentar a velocidade do cálculo, nós congelaremos a superfície e permitiremos o adsorvato de relaxar.

In [80]:
constraint = FixAtoms(mask=[atom.symbol=='Ag' for atom in ads_system])
ads_system.set_constraint(constraint)

In [81]:
view(ads_system, viewer='x3d')

In [82]:
# now we set calculation for the adsorbed system
calc.set(directory='adsorcao/Ag-slab-O-fcc',
        xc='PBE',
        kpts=[4, 4, 1],
        encut=450,
        ibrion=2,
        atoms=ads_system)

In [83]:
E_fcc = ads_system.get_potential_energy()

print("Energia do sistema Ag(111) + O (fcc) = {:.3f} eV".format(E_fcc))

Energia do sistema Ag(111) + O (fcc) = -35.108 eV


Calculando energia de adsorção 

$(1/2) O_2 + Ads \to O-Ads$

In [84]:
Hads_fcc = E_fcc - E_slab - 0.5 * E_O2

print("Energia de adsorção do O na posição fcc do slab Ag(111) = {:.3f} eV".format(Hads_fcc))

Energia de adsorção do O na posição fcc do slab Ag(111) = -0.380 eV


## 2. Átomo de O na posição Bridge

In [85]:
calc = Vasp(restart=True, directory='adsorcao/Ag-fcc111-slab')

ads_system = fcc111("Ag", size=(2,2,3), vacuum=10.0,periodic=True)
ads_system.calc = calc

# adding O in bridge site
add_adsorbate(ads_system, "O", height=1.2, position="bridge")

# now we constrain the slab
constraint = FixAtoms(mask=[atom.symbol=='Ag' for atom in ads_system])
ads_system.set_constraint(constraint)

# now we set calculation for the adsorbed system
calc.set(directory='adsorcao/Ag-slab-O-bridge',
        xc='PBE',
        kpts=[4, 4, 1],
        encut=450,
        ibrion=2,
        atoms=ads_system)

In [87]:
view(ads_system, viewer='x3d')

In [86]:
E_bridge = ads_system.get_potential_energy()

print("Energia do sistema Ag(111) + O (bridge) = {:.3f} eV".format(E_bridge))

Energia do sistema Ag(111) + O (bridge) = -34.298 eV


Demorou 2m 26s

In [88]:
Hads_bridge = E_bridge - E_slab - 0.5 * E_O2

print("Energia de adsorção do O na posição bridge do slab Ag(111) = {:.3f} eV".format(Hads_bridge))

Energia de adsorção do O na posição bridge do slab Ag(111) = 0.430 eV


## 3. Átomo de O na posição HCP

In [89]:
calc = Vasp(restart=True, directory='adsorcao/Ag-fcc111-slab')

ads_system = fcc111("Ag", size=(2,2,3), vacuum=10.0,periodic=True)
ads_system.calc = calc

# adding O in bridge site
add_adsorbate(ads_system, "O", height=1.2, position="hcp")

# now we constrain the slab
constraint = FixAtoms(mask=[atom.symbol=='Ag' for atom in ads_system])
ads_system.set_constraint(constraint)

# now we set calculation for the adsorbed system
calc.set(directory='adsorcao/Ag-slab-O-hcp',
        xc='PBE',
        kpts=[4, 4, 1],
        encut=450,
        ibrion=2,
        atoms=ads_system)

In [90]:
view(ads_system, viewer='x3d')

In [91]:
E_hcp = ads_system.get_potential_energy()

print("Energia do sistema Ag(111) + O (hcp) = {:.3f} eV".format(E_hcp))

Energia do sistema Ag(111) + O (hcp) = -35.010 eV


Demorou 1m 76s

In [92]:
Hads_hcp = E_hcp - E_slab - 0.5 * E_O2

print("Energia de adsorção do O na posição hcp do slab Ag(111) = {:.3f} eV".format(Hads_hcp))

Energia de adsorção do O na posição hcp do slab Ag(111) = -0.282 eV


Comparando as energias de adsorção

In [93]:
print("Comparando as energias de adsorção")
print("Posição fcc:  {:.3f} eV".format(Hads_fcc))
print("Posição bridge: {:.3f} eV".format(Hads_bridge))
print("Posição hcp:  {:.3f} eV".format(Hads_hcp))

Comparando as energias de adsorção
Posição fcc:  -0.380 eV
Posição bridge: 0.430 eV
Posição hcp:  -0.282 eV


# Adsorção de moléculas mais complicadas

Criando a molécula de benzeno 

In [150]:
from ase.build import molecule

# criando a molécula de benzeno
benzene = molecule("C6H6")
benzene.center(vacuum=5)
benzene.pbc = True

calc = Vasp(directory='adsorcao/benzeno',
            xc="PBE",
            encut=450,
            kpts=[1, 1, 1],
            ibrion=2,
            nsw=100,
            ivdw=12,                  # Adicionando correção de van der Waals D3(BJ) (molécula grande)
            atoms=benzene)

E_bezeno = benzene.get_potential_energy()

Demorou 25.3 s

In [151]:
print("Energia do benzeno = {:.3f} eV".format(E_bezeno))

Energia do benzeno = -76.290 eV


Criando o slab de Ag(111)

In [152]:
# criando um slab Ag(111)
slab = fcc111("Ag", size=(3,3,3), vacuum=10.0,periodic=True)

# now we constrain the slab
constraint = FixAtoms(mask=[atom.symbol=='Ag' for atom in ads_system])
ads_system.set_constraint(constraint)

calc = Vasp(directory='adsorcao/Ag111-slab-vdw',
            xc='PBE',
            kpts=[4,4,1],  
            encut=450,
            ivdw=12,                  # Adicionando correção de van der Waals D3(BJ) (molécula grande)
            atoms=slab)

E_slab_vdw = slab.get_potential_energy()

Demorou 3m 53s 

In [153]:
print("Energia do slab Ag(111) = {:.3f} eV".format(E_slab_vdw))

Energia do slab Ag(111) = -78.278 eV


Criando o sistema de adsorção

In [154]:
# criando o sistema de adsorção
ads_system = fcc111("Ag", size=(3,3,3), vacuum=10.0,periodic=True)

benz = molecule("C6H6")
benz.center(about=ads_system.get_positions()[-9:].mean(axis=0)) # centraliza o benzeno na superfície do slab
benz.translate([0.0,0.0,2.5])  # altura de 2.5 Å do slab
ads_system += benz

# now we constrain the slab
constraint = FixAtoms(mask=[atom.symbol=='Ag' for atom in ads_system])
ads_system.set_constraint(constraint)

In [155]:
view(ads_system, viewer='x3d')

In [156]:
calc = Vasp(directory='adsorcao/benzene-Ag111-center',
            xc='PBE',
            kpts=[4,4,1],  
            encut=450,
            ivdw=12,                  # Adicionando correção de van der Waals D3(BJ) (molécula grande)
            ibrion=2,
            nsw=100,
            lvtot=True,                # keep LOCPOT for post-processing
            atoms=ads_system)

E_ads_center = ads_system.get_potential_energy()

Demorou 106min 52s

In [157]:
print("Energia do sistema Ag(111) + benzeno (center) = {:.3f} eV".format(E_ads_center))

Energia do sistema Ag(111) + benzeno (center) = -155.250 eV


Energia do sistema Ag(111) + benzeno (center) = -155.250 eV

In [158]:
Hads_benz_center = E_ads_center - E_slab_vdw - E_bezeno

print("Energia de adsorção do benzeno na posição center do slab Ag(111) = {:.3f} eV".format(Hads_benz_center))

Energia de adsorção do benzeno na posição center do slab Ag(111) = -0.683 eV


Energia de adsorção do benzeno na posição center do slab Ag(111) = -0.683 eV

In [159]:
view(ads_system, viewer='x3d')

In [188]:
ads_system.get_chemical_symbols()[26:28]

['Ag', 'C']

In [191]:
c_benzene = ads_system.get_positions()[27]

slab_top = ads_system.get_positions()[26]

print("Distância do benzeno até o topo do slab: {:.3f} Å".format(c_benzene[2]-slab_top[2]))

Distância do benzeno até o topo do slab: 3.110 Å


Distância do benzeno até o topo do slab: 3.110 Å